<img src="images/cflow.png" width="300" height="500"/>

***

The `-c` flag tells gcc to compile the program and generate machine code, but not to link it or generate an executable:

```
$ gcc hello.c -c
```
The result is a file named `hello.o`, where the `o` stands for “object code”, which is the compiled program. Object code is not executable, but it can be linked into an executable.

Also note below - 

```
c:\C>nm hello.o
00000000 b .bss
00000000 d .data
00000000 r .eh_frame
00000000 r .rdata
00000000 r .rdata$zzz
00000000 t .text
         U ___main
00000000 T _main
         U _printf
         U _scanf
```

The UNIX command `nm` reads an object file and generates information about the names it defines and uses. (This command works on Windows as well through cygwin)

You can control how much optimization gcc does with the `-O` flag. By default, it does very little optimization, which can make debugging easier.

The option `-O1` turns on the most common and safe optimizations. Higher numbers turn on additional optimizations that require longer compilation time.

Similar to the `-c` flag, the `-S` flag tells gcc to compile the program and generate assembly code.

Taking another step backward through the compilation process, you can use the `-E` flag to run the preprocessor only:

    $ gcc hello.c -E

The result is the output from the preprocessor. In this example, it contains the included code from `stdio.h`, and all the files included from stdio.h, and all the files included from those files, and so on. On my machine, the total is more than 800 lines of code.

#### C – data types:

There are four data types in C language. They are,

Types|	Data Types
-----|-----------|
Basic data types|	int, char, float, double
Enumeration data type|	Enum
Derived data type|	pointer, array, structure, union
Void data type|	Void


#### Integer data types

Name|Typical Size|Signed by default?|
---|------------|-------------------|
char|1 byte| compiler dependent, usually signed|
short|2 bytes| signed|
int|4 bytes|signed|
long|4 bytes|singed|
long long| 8 bytes|signed|

Note – If we use `int` data type to store decimal values, decimal values will be truncated and we will get only whole number. 


### Integer Overflow

The behaviour when integer overflow occurs depends on whether the operands were signed or unsigned. When overflow occurs during an operation on *signed* integers, the program's behabiour is undefined. 

When overflow occurs an operaion on unsigned integers, the result *is* defined: we get the correct answer modulo $2^n$, where $n$ is the number of bits used to store the result. For example, if we add 1 to the unsigned 16-bit number 65,535, the result is guaranteed to be 1. 

#### Modifiers in C language:
 
 - The amount of memory space to be allocated for a variable is derived by modifiers.
 - Modifiers are prefixed with basic data types to modify (either increase or decrease) the amount of storage space allocated to a variable.
 - For example, storage space for int data type is 4 byte for 32/64 bit processor. We can increase the range by using long int which is 8 byte. We can decrease the range by using short int which is 2 byte.
 - There are 5 modifiers available in C language. They are, 
1.	short
2.	long
3.	signed
4.	unsigned
5.	long long



### Storage Classes

Every variable in C programming has two properties: type and storage class. Type refers to the data type of a variable. And, storage class determines the scope and lifetime of a variable. There are 4 types of storage class:

1.	automatic (local)
2.	external (global)
3.	static
4.	register (outdated)

Local variables are defined inside the functions and can’t be accessed from outside. Global variables are defined outside of functions and can be accessed by any function.

Another illustration of local variables

```c
int frotz(int a)
{
int b;
b = 10; /* in scope (from the local definition) */
a = 20; /* in scope (from the parameter list) */
c = 30; /* ERROR, out of scope (declared in another block, in main()) */
}
int main(void)
{
int c;
c = 20; /* in scope */
b = 30; /* ERROR, out of scope (declared above in frotz()) */
return 0;
}
```

Thre are a few variable declaration keywords commonly used in C that do not specify variable types, but a related concept called *storage classes*. Two common examples of storage class specifiers are `extern` and `static`.

#### External variable

Sometimes a C program consists of multiple source files. In such cases, there may be scenarios in which we may need to use the variable in defined in other files. We can use the global variables in files other than the file in which they were created by redeclaring it, prefixed by the specifier `extern`, in the other files. 

```c
//file main.c

int main()
{
extern int my_var;
print_value();      //will print 11
return 0;
}

//file secondary.c

#include <stdio.h>

int my_var = 11;

void print_value()
{
printf("my variable is %d\n", my_var);    
}

```




In above example, the variable `my_var` is defined in `secondary.c` and assigned a value in `main.c`. The value of
`my_var` is printed out by calling function `print_value` in `main.c` which was defined in `secondary.c`.

The above progam is split in 2 source files which can be compiled to produce one executables by issuing following command `gcc -o testprogram main.c secondary.c`.







#### Static Variables

A second important storage class classifier is `static`. Normally, when we call a function, all its local variables are reinitialized each time the function is called. This means that their values change between function calls. Static variables, however, maintain their values between function calls. 

In [43]:
#include <stdio.h>
void display()
{
    static int c = 0;
    printf("%d  ",c);
    c += 5;
};

int main()
{
for (int i=0; i<4; i++)    
    {display();}
};

0  5  10  15  

Every global variable is defined as `static` automatically, that is, its value persists during the life of a program. 


At the global scope, `static` means that the variable or function declared `static` is *only visible in this particular source file*, and cannot be referenced from other source files. Again, this definition of `static` only pertains to the global scope. `static` still means the same old thing in the local scope of the function.

See following example - 

```c
//foo.c

int foo=11;
static void *pointer_array[10]; /* array of pointers to now no one can see it except this file! */
static int index=0; /* same for this one! */
/* but these functions are NOT static, */
/* so they can be used from other files: */
void append_pointer(void *p)
{
    pointer_array[index++] = p;
}
void *get_pointer(int i)
{
    return pointer_array[i];
}

//bar.h

//put function prototypes here
void append_pointer(void *p);
void *get_pointer(int i);

//baz.c

# include <stdio.h>
#include "bar.h"

extern int foo;
int main(void)
{
char *s = "some data!";  /* s points to a constant string (char*) */
int a = 10;
int *b;
    
char *s2;  /* when we call get_pointer(), we'll store them back here */
int *b2;

b = &a; /* b is a pointer to a */
    
/* now let's store them both, even though they're different types */

append_pointer(s);
append_pointer(b);

/* they're stored!  let's get them back! */
    
s2 = get_pointer(0); /* this was at index 0 */
b2 = get_pointer(1); /* this was at index 1 */

printf("s2  = %p\nb2 =%p\n", s2, b2);
printf("value of foo is %d\n", foo);
return 0; 
}
```

We created a file `foo.c`. In this file, we declared a normal `int` variable, 2 `static` variables and also defined 2 functions. Then we created a `bar.h` header file which has function prototypes of the functions we defined in `foo.c`. Finally, we created `baz.c` in which we call the functions defined in `foo.c` and also access the variable defined in `foo.c`.

We compile these files by issuing command `gcc -o testprogram foo.c baz.c`. When `testprogram` was run, it produced the following outputs -
```
s2  = 0x102846f86
b2 =0x7ffeed3b9a7c
value of foo is 11
```

This example teaches us 4 things -

 - 1 Use of `static` global variables
 - 2 Use of `extern` variable
 - 3 Use of custom header file
 - 4 How to compile a program consisting of multiple source files.

In [1]:
#include<stdio.h>
int main(void)
{
    char ch = 'A';
    char str[20] = "mayank gupta";
    float flt =  10.234;
    int no =  150;
    double dbl = 20.123456;
    printf("Character is %c \n", ch);
    printf("String is %s \n", str);
    printf("Float value is %f \n", flt);
    printf("Integer value is %d \n", no);
    printf("Double value is %lf \n", dbl);
    printf("Octal value is %o \n", no);
    printf("Hex value is %x \n", no);
}

Character is A 
String is mayank gupta 
Float value is 10.234000 
Integer value is 150 
Double value is 20.123456 
Octal value is 226 
Hex value is 96 


In [44]:
#include <stdio.h>

int main(void)
{
char s[10] = "mayank g";
printf(s);
}

/var/folders/d4/z0x5s3410lj8px16wr2m5y200000gp/T/tmpk0c0vskg.c:6:8: warning: format string is not a string literal (potentially insecure) [-Wformat-security]
printf(s);
       ^
/var/folders/d4/z0x5s3410lj8px16wr2m5y200000gp/T/tmpk0c0vskg.c:6:8: note: treat the string as an argument to avoid this
printf(s);
       ^
       "%s", 
1 warning generated.


mayank g

**Note -** `printf` function returns the count of character printed. See next example.

In [10]:
#include <stdio.h>

int i =  1;
int main(void)
{
	int i = printf("Hello World!\n");
    printf("%d\n", i);
	return 0;	
}

Hello World!
13


### `scanf`

In [16]:
// run this on command prompt. The C kernel in notebook is not capable of taking inputs.
# include <stdio.h>

int main(void)
{
	char ch;
	char str[100];
	printf("Enter any character \n");
	scanf("%c", &ch);
	printf("Entered character is %c \n", ch);
	printf("Enter any string (upto 100 ch)\n");
	scanf("%s", &str);
	printf("Entered string is %s \n", str);
    return 0;
}


/var/folders/d4/z0x5s3410lj8px16wr2m5y200000gp/T/tmp3yqq574p.c:12:14: warning: format specifies type 'char *' but the argument has type 'char (*)[100]' [-Wformat]
        scanf("%s", &str);
               ~~   ^~~~
1 warning generated.


Enter any character 
Entered character is   
Enter any string (upto 100 ch)
Entered string is  


In [ ]:
#include <stdio.h>

main() /*rudimentary calculator from KR page 158*/
{
	double sum, v;
	sum  = 0;
	while (scanf("%lf", &v) == 1)
		printf("\t%.2f\n", sum += v);
	return 0;
}

/* o/p
11
	11.00
13
	24.00
*/    

Note in above program how `scanf` has been compared to 1. Basically `scanf` function returns integer value corresponding to characters successfully scanned. 


### `getchar()`

```C
main()
{
	long nc;
	nc = 0;
	while(getchar() != EOF)
		nc=nc+1;
	printf("%ld\n",nc);			
}                            //keystroke for EOF is CTRL+Z
```

Alternate way

```C
#include <stdio.h>
main()
{
	double nc;
	for (nc =0; getchar() != EOF; ++nc)
		;                             // notice that we used ; without any statement. 
	printf("%.1f\n", nc); 
 }
                      
// .1f means 1 digit after decimal point. To suppress decimal point, use .0f
```

### `putchar()`

Notes from K&R –

>A character written between single quotes represents an integer value equal to the numerical value of the character in the machine's character set. This is called a character constant, although it is just another way to write a small integer. So, for example, 'A' is a character constant; in the ASCII character set its value is 65, the internal representation of the character A. Of course, 'A' is to be preferred over 65: its meaning is obvious, and it is independent of a particular character set. 

>The escape sequences used in string constants are also legal in character constants, so '\n' stands for the value of the newline character, which is 10 in ASCII. You should note carefully that '\n' is a single character, and in expressions is just an integer; on the other hand, “\n” is a string constant that happens to contain only one character. 

In [1]:
#include <stdio.h>

int main(){
char i;
for (i= 20; i <35; ++i){
    printf("%d - %c\n",i,i); 

//putchar(i) followed by putchar('\n') can be used above

};

printf("for loop ends here\n");

putchar('A'+10);     //o/p is K as ‘A’ has ascii value of 65 and 65+10=75 which is ascii value of ‘K’
putchar('\n');    
putchar('A'+'1');    // o/p=’r’ as ‘A’ has ascii value of 65. 65+49=114 which is ascii value of ‘r’
putchar('\n');
putchar('A'+'\n');   // o=K as ‘A’ has asci value of 65 and 65+10(‘\n’)=75 which is ascii value of K

return 0;
}

 //"" can't be used in putchar('\n').

20 - 
21 - 
22 - 
23 - 
24 - 
25 - 
26 - 
27 - 
28 - 
29 - 
30 - 
31 - 
32 -  
33 - !
34 - "
for loop ends here
K
r
K

In [ ]:
#include <stdio.h>

int main(){
char i;
for (i= 125; i <128; ++i){
    printf("%d - %c\n",i,i); 
};
return 0;
}

/var/folders/d4/z0x5s3410lj8px16wr2m5y200000gp/T/tmpizjsfutt.c:5:16: warning: result of comparison of constant 128 with expression of type 'char' is always true [-Wtautological-constant-out-of-range-compare]
for (i= 125; i <128; ++i){
             ~ ^~~~
1 warning generated.


Notice the warning message produced by compiler. Also read following sections.

In [8]:
#include <stdio.h>

int main(){
	int i;
	for(i=0; i <128; ++i){
	printf("for i=%d, c is %c\n", i,i);
	}
}
/*
Output will be different on Windows machine for some initial characters.
*/

for i=0, c is  
for i=1, c is 
for i=2, c is 
for i=3, c is 
for i=4, c is 
for i=5, c is 
for i=6, c is 
for i=7, c is 
for i=8, c is 
for i=9, c is 	
for i=10, c is 

for i=11, c is 
for i=12, c is 
for i=13, c is 
for i=14, c is 
for i=15, c is 
for i=16, c is 
for i=17, c is 
for i=18, c is 
for i=19, c is 
for i=20, c is 
for i=21, c is 
for i=22, c is 
for i=23, c is 
for i=24, c is 
for i=25, c is 
for i=26, c is 
for i=27, c is 
for i=28, c is 
for i=29, c is 
for i=30, c is 
for i=31, c is 
for i=32, c is  
for i=33, c is !
for i=34, c is "
for i=35, c is #
for i=36, c is $
for i=37, c is %
for i=38, c is &
for i=39, c is '
for i=40, c is (
for i=41, c is )
for i=42, c is *
for i=43, c is +
for i=44, c is ,
for i=45, c is -
for i=46, c is .
for i=47, c is /
for i=48, c is 0
for i=49, c is 1
for i=50, c is 2
for i=51, c is 3
for i=52, c is 4
for i=53, c is 5
for i=54, c is 6
for i=55, c is 7
for i=56, c is 8
for i=57, c is 9
for i=58, c is :
for i=59

In [23]:
#include <stdio.h>
#include <limits.h>

int main(){
printf("%d, %d", CHAR_MIN, CHAR_MAX);
}

-128, 127

In [25]:
#include <stdio.h>
#include <limits.h>

int main(){
printf("%d, %d", INT_MIN, INT_MAX);
}

-2147483648, 2147483647

'The C standard does not specify if plain `char` is signed or unsigned. 

In fact, the standard defines three distinct types: `char`, `signed char`, and `unsigned char`. If you `#include <limits.h>` and then look at `CHAR_MIN` (see above program), you can find out if plain `char` is `signed` or `unsigned`, but even then, the three types are distinct as far as the standard is concerned. See [this SO](..) for more details. 

In [5]:
#include <stdio.h>

int main(){
    int d;
    d = 'A' + 8; //65+8,because 65 is ASCII value for 'A'
    printf("%d\n",d);  
    printf("%d",'A'); // ascii value for 'A'

}

73
65

In [9]:
#include <stdio.h>

int main(){
    if('A'> 'r' || 'A' > 70) // See how ‘OR’ has been implemented here.
        printf("A\n");
else
    printf("B\n");    
}


B


Another example above.


Notice how characters have been compared. ‘A’ is smaller than ‘r’ because asci value of ‘A’ is 65 which is smaller than the ascii value of ‘r’. Also 65 is less than 70 so both conditions fail and output is ‘B’. 


Note - the compiler "promotes" the smaller type (`char`) to be the same size as the larger type (`int`) before combining the values. Promotions are determined at compile time based purely on the types of the values in the expressions. Promotions do not lose information -- they always convert from a type to compatible, larger type to avoid losing information. For example, division such as 3.0/2. Here 2 will be promoted to float. 

The opposite of promotion, truncation moves a value from a type to a smaller type. In that case, the compiler just drops the extra bits. 
Assigning from an integer to a smaller integer (e.g.. `long` to `int`, or `int` to `char`) drops the most significant bits. Assigning from a floating point type to an integer drops the fractional part of the number.




In [10]:
#include <stdio.h>

int main(){
	char ch;
	int i;
	i = 321;
	ch = i;
	printf("%d", ch);
	}


65


The assignment will drop the upper bits of the `int` 321. The lower 8 bits of the number 321 represents the number 65 (321 - 256). So the value of `ch` will be (`char`)65 which happens to be 'A'. The assignment of a floating point type to an integer type will drop the fractional part of the number. The following code will set `i` to the value 3. This happens when assigning a floating point number to an integer or passing a floating point number to a function which takes an integer.
```
double pi;
int i;
pi = 3.14159;
i = pi; // truncation of a double to fit in an int
// i is now 3
```


In [1]:
int main(void)
{
char i = 127;
printf("%d - %c",i, i);
}

/var/folders/d4/z0x5s3410lj8px16wr2m5y200000gp/T/tmpirqrpvkw.c:4:1: warning: implicitly declaring library function 'printf' with type 'int (const char *, ...)' [-Wimplicit-function-declaration]
printf("%d - %c",i, i);
^
/var/folders/d4/z0x5s3410lj8px16wr2m5y200000gp/T/tmpirqrpvkw.c:4:1: note: include the header <stdio.h> or explicitly provide a declaration for 'printf'
1 warning generated.


127 - 

See above program carefully because something interesting happened here. 

First thing to keep in mind is that `printf` function is not the part of standard C language. To use it, we include the header `stdio.h`. I forgot to include the header `stdio.h` which is needed to use the `printf` function but compiler still went ahead to compile the program, although it did warn us about the mistake.

### `gets() and puts()`
```C
#include <stdio.h>

main()
{
	char name[50];
	printf("Enter your name\n");
	gets(name);
	printf("Your name is:");
	puts(name);
}
```

#### `sizeof()` operator in C language:

```C
#include <stdio.h>
#include <limits.h>

void main()
{
	int a; char b; float c; double d;
	printf("Storage size for int data type: %d\n", sizeof(a));
	printf("Storage size for char data type: %d\n", sizeof(b));
	printf("Storage size for float data type: %d\n", sizeof(c));
	printf("Storage size for double data type: %d\n", sizeof(d));
}
Storage size for int data type: 4
Storage size for char data type: 1
Storage size for float data type: 4
Storage size for double data type: 8
//above program was run on 32 bit Windows machine. 
```


In [27]:
#include <stdio.h>
#include <limits.h>

int main()
{
    int a; char b; float c; double d; long int e;
    printf("Storage size for int data type: %lu\n", sizeof(a));
    printf("Storage size for char data type: %lu\n", sizeof(b));
    printf("Storage size for float data type: %lu\n", sizeof(c));
    printf("Storage size for double data type: %lu\n", sizeof(d));
    printf("Storage size for long int data type: %lu\n", sizeof(e));
}

//above program was run on 64 bit Mac machine. 

Storage size for int data type: 4
Storage size for char data type: 1
Storage size for float data type: 4
Storage size for double data type: 8
Storage size for long int data type: 8


### `if` clause

```c
#include <stdio.h>

main()

{
	int i = 10;
	if(1)
		printf("Hi there\n");
	printf("True hence %d\n", i == 10);
	printf("False hence %d\n", i > 20);
}
```
Notice how `if(1)` evaluates to `true`, also notice that first `printf` prints `1` while second print `0`.


### `for` loop (infinite loop with ‘`break`’)
```c
#include <stdio.h>

main()
{
	int i;
	
	for(;;){
		printf("enter some number\n");
	    scanf("%d", &i);
		if(i == 3)	
			break;
	}
		printf("program terminated");		
}
```
Notice that all three components of `for` loop has been omitted, thereby making this ‘infinite loop’,presumably to be broken by other means, such as `break` or `return`.
Note – while writing this program, I forgot to include ‘{}’ in for loop, due to this program didn’t compile. If there is single expression, this can be omitted. Otherwise, it must be included. 


### `while` loop

In [5]:
#include <stdio.h>
int main(void){
    int i; 
    i = 0;
    while(i-10){
        printf("%d", i);
        i++;
        if (i == 8) 
            break;
    }} // o/p -> 1234567

01234567



If `i` was tested for equality with 12, instead of 8, we would have got 123456789 as output. Notice, how `break` works here.

```c
#include <stdio.h>
//continue use
void main(){
	int i; 
	i = - 1;
	while(i-10){
		i++;
		if (i == 7)
			continue;
		printf("%d\n", i);				
	}}
 o/p – 01234568910 //each digit in new line. Notice 7 isn’t printed.
```

C does not have a distinct boolean type-- `int` is used instead. The language treats integer 0 as `false` and all non-zero values as `true`. So the statement...
```
i = 0;
while (i - 10) {
...
```
will execute until the variable `i` takes on the value 10 at which time the expression `(i -10)` will become false (i.e. 0) in above example.


#### `do -  while` loop

```C
#include <stdio.h>

int main()
{
  int x = 1;
  do {
    /* "Hello, world!" is printed at least once even though the condition is false */
      printf( "Hello, world!\n" );
	  x++;
	  printf("%d\n", x);
  } while ( x <0 );
}
```
```
o/p- 
Hello World!
2 // ‘do-while’executes statements at least once.
```


Another example from **KR** -

```c
#include <stdio.h>

int strlen(char *s)
{
	int n;
	
	for (n = 0; *s != '\0'; s++)
{
		n++;
		printf("%d\n",s); /*this is not needed. Just to see what gets printed for s*/
	}
	return n;
}
main()
{
	char str[26];
	printf("Enter some string\n");
	scanf("%s", &str);
	printf("length of string is %d", strlen(str));
}	


/*above program doesn't work for something like "maya g" (inverted comma not part of string) because 'for' loop conditon fails when it encounter space. 
/*From KR - as formal parameters in a function definition,
char s[];
and
char *s;

are equivalent.*/
```

	

In for loop above, we used the condition  `*`s != ‘\0’. In reference to this, a note from KR (page 38) – 

The character constant ‘\0’ represents the character with zero value, the null character.’\0’ is often written instead of 0 to emphasize the character nature of some expression, but the numeric value is just 0.

Also,from page 39 – 

>Be careful to distinguish between a character constant and a string that contains a single character: `‘x’` is not the same as `“x”`. The former is an integer, used to produce the numeric the value of the letter x in the machine’s character set. The latter is an array of characters that contains one character (the letter x) and a `\0`.

In [41]:
#include <stdio.h>
#include <stdlib.h>

int G = 0; /*global var*/

int main(int argc, char **argv)
{
    static int s;  
    int a;
    int *p;

    p = malloc(sizeof(int));

    printf("&G  = %p\n", (void *) &G);
    printf("&s  = %p\n", (void *) &s);
    printf("&a  = %p\n", (void *) &a);
    printf("&p  = %p\n", (void *) &p);
    printf("p   = %p\n", (void *) p);            /*notice ‘p’ format specifier*/
    //printf("p  = %d\n", (void *) p);          /*notice ‘d’format specifier*/
    printf("main  = %p\n", (void *) main); 

    free(p);

}

&G  = 0x10c531028
&s  = 0x10c53102c
&a  = 0x7ffee36e388c
&p  = 0x7ffee36e3880
p   = 0x7f921fc02c80
main  = 0x10c530e60


Note that in some cases, we might see -ve decimal values for addresses. Addresses aren't always positive - on x86_64, pointers are sign-extended and the address space is clustered symmetrically around 0. [SO Post](https://stackoverflow.com/questions/3304795/can-a-pointer-address-ever-be-negative)

The sign of the number depends on the highest bit set (assuming two's complement representation, which is used by the vast majority of systems currently in use), so a memory address above `0x80000000` on a 32-bit system will be negative, and a memory address below `0x80000000` will be positive. There's no real significance to that.

You should be printing memory addresses using the `%p` modifier; alternatively, some people use `%08x` for printing memory addresses (or `%016llx` on 64-bit systems). This will always print out as an unsigned integer in hexadecimal, which is far more useful than a signed decimal integer. [SO Post](https://stackoverflow.com/questions/1689423/memory-address-positive-or-negative-value-in-c)

***

##### Memory segment 

This part is from Think OS book.

The data of a running process is organized into 5 segments:

• The code segment contains the program text; that is, the machine language instructions that make up the program.

• The static segment contains immutable values, like string literals. For example, if your program contains the string "Hello, World", those character will be stored in the static segment.

• The globals segment contains global variables and local variables that are declared static.

• The heap segment contains chunks of memory allocated at run time, usually by calling the C library function malloc.

• The stack segment contains the run-time stack, which is made up of stack frames. Each time a function is called, a stack frame is allocated to contain the parameters and local variables of the function.
The arrangement of these segments is determined partly by the compiler and partly by the operating system. The details vary from one system to another, but in the most common arrangement:

• The text segment is near the “bottom” of memory, that is, at addresses near 0.

• The static segment is often just above the text segment, that is, at higher addresses.

• The global segment is often just above the static segment.

• The heap is often above the static segment. As it expands, it grows up toward larger addresses.

• The stack is near the top of memory; that is, near the highest addresses in the virtual address space. As the stack expands, it grows down toward smaller addresses. 

To determine the layout of these segments on your system, try running this program -

`main` is the name of a function; when it is used as a variable, it refers to the address of the first machine language instruction in `main`, which we expect to be in the text segment. `global` is a global variable, so we expect it to be in the global segment. `local` is a local variable, so we expect it to be on the stacks refers to a “string literal", which is a string that appears as part of the program (as opposed to a string that is read from a file, input by a user, etc.). We expect the location of the string to be in the static segment (as opposed to the pointer, `s`, which is a local variable). `p` contains an address returned by `malloc`, which allocates space in the heap. `malloc` stands for “memory allocate.” The format sequence `%p` tells printf to format each address as a “pointer”, so it displays the results in hexadecimal. When I run this program, the output looks like this (I added spaces to make it easier to read):

As expected, the address of main is the lowest, followed by the location of the string literal. The location of global is next, then the address p points to. The address of local is much bigger. The largest address has 12 hexadecimal digits. Each hex digit corresponds to 4 bits, so it is a 48-bit address. That suggests that the usable part of the virtual address space is 248 bytes.


#### Unary Increment Operators: ++ --

The unary ++ and -- operators increment or decrement the value in a variable. There are "pre" and "post" variants for both operators which do slightly different things (explained below)

var++ increment "post" variant
++var increment "pre" variant
var-- decrement "post" variant
--var decrement "pre" variant
```c
int i = 42;
i++; // increment on i
// i is now 43
i--; // decrement on i
// i is now 42
```
#### Pre and Post Variations

The Pre/Post variation has to do with nesting a variable with the increment or decrement operator inside an expression -- should the entire expression represent the value of the variable before or after the change? I never use the operators in this way (see below), but an example looks like...
```c
int i = 42;
int j;
j = (i++ + 10);
// i is now 43
// j is now 52 (NOT 53)
j = (++i + 10)
// i is now 44
// j is now 54
```

***

#### Conditional Expression -or- The Ternary Operator

The conditional expression can be used as a shorthand for some if-else statements. The general syntax of the conditional operator is:

`<expression1> ? <expression2> : <expression3>` This is an expression, not a statement, so it represents a value. The operator works by evaluating `expression1`. If it is true (non-zero), it evaluates and returns `expression2`. Otherwise, it evaluates and returns expression3.
The classic example of the ternary operator is to return the smaller of two variables. Every once in a while, the following form is just what you needed. Instead of...
```
if (x < y) {
min = x;
}
else {
min = y;
}
12
```
You just say - `min = (x < y) ? x : y;`


***

#### Parameters to `main` function

```C
#include <stdio.h>
#include <stdlib.h>

int SumRange(int start, int end)
{
	int i;
	int sum;
	sum = 0;
	for(i = start; i <end; i++){
		sum += i;
	}
	return sum;
	
}

int main(int argc, char **argv)
{
	int start;
	int end;
	if(argc !=3){
		fprintf(stderr, "Usage: %s\n start end", argv[0]);
		return 1;
	}
	
start = atoi(argv[1]);
end = atoi(argv[2]);

printf("SumRange(%d, %d) = %d\n", start, end, SumRange(start,end));
return 0;
}
```

---
In above program notice that how `main` function got 2 parameters, namely, `argc` animad `argv`. Also notice the use of function `atoi` from `stdlib` library. 

Suppose there is a program ‘echo’ which echoes its commandline arguments on a single line, separated by blanks. That is the command

    echo hello, world

prints the output   

    hello, world

For this, we pass two parameters to ‘main’ function, namely `argc` and `*argv[]`. Per standard, `argv[argc]` has to be a null pointer. See illustration below.


![](Images/argv.png)


`argv` is a pointer to an array of character strings that contain the arguments. An example from KR –


```C
clarg.c 
#include <stdio.h>

main (int argc, char *argv[])
{
	int i;
	for (i =1; i < argc; i++)
		printf("%s%s", argv[i], (i < argc-1) ? " " : ""); /*see this carefully*/
	printf("\n");
	return 0;
}
```
```
c:\C\KR>clarg 1 2
1 2
```

Since `argv` is a pointer to an array of pointers, we can manipulate the pointer rather than index the array. See the another version of above program – (Read page 114-115 from KR to better understand above and below example)

```C
#include <stdio.h>

main (int argc, char *argv[])
{
	while(--argc > 0)
		printf("%s%s", *++argv, (argc > 1) ? " " : "");
	printf("\n");
	return 0;
}
```


In [3]:
#include <stdio.h>
main()
{
char s[4];
scanf("%s", &s);
printf("%s \n", s);
}


/var/folders/d4/z0x5s3410lj8px16wr2m5y200000gp/T/tmp6bkeq2_m.c:2:1: warning: type specifier missing, defaults to 'int' [-Wimplicit-int]
main()
^
/var/folders/d4/z0x5s3410lj8px16wr2m5y200000gp/T/tmp6bkeq2_m.c:6:13: warning: format specifies type 'char *' but the argument has type 'char (*)[4]' [-Wformat]
scanf("%s", &s);
       ~~   ^~
2 warnings generated.


I am providing following random inputs for above program-


#1st input

`abcdeffdfsd`

#o/p->    

`abcdeffdfsd`

#2nd input

`afsdfsfsfasfasffasfasfsfafafsdfsadfasfsadfsdfasfasffsdfasfasfsfasdfasfafasf`
#o/p->

`afsdfsfsfasfasffasfasfsfafafsdfsadfasfsadfsdfasfasffsdfasfasfsfasdfasfafasf 
Segmentation fault: 11
`

Although I declared array `s` of length 4, it nevertheless was able to print string of length exceeding 4 in first case and it produces segment fault in second case. 

`scanf()` doesn't do any bounds checking. It will happily fill an array past its bounds clobbering whatever was in memory after the array. In first example nothing appears to break but we've definitely written into memory we weren't supposed to. In second example we've written into memory pass the array and when we tried to read it it just so happened to trigger the OS to be like, "No, you can't do that. Next time behave yourself." What we witnessed is an example of C's undefined behaviour. See [this](https://www.reddit.com/r/learnprogramming/comments/g6lu0n/beginners_confusion_about_array_in_c/) and [this](https://stackoverflow.com/questions/10051782/array-overflow-why-does-this-work).



See another related example below -

In [ ]:
#include <stdio.h>

int main(void)
{
  char str[5] = "hello";
  puts(str);
}

Although this compiles without any warning but it is an incorrect program. The reason is that, in C, every string is null terminated. The null terminator is the symbol value 0 (zero). It is used to mark the end of a string. This is necessary since the size of the string isn't stored anywhere.

Therefore, every time you allocate room for a string, you must include sufficient space for the null terminator character. Your example does not do this, it only allocates room for the 5 characters of "hello". Correct code should be:
```c
char str[6] = "hello";
```
You can also omit the array size altogether. That is, we can also write - 

```c
char str[] = "hello";
```

In the last case the size of the character array is determined from the number of initializers of the string literal that is equal to 6.

#### Points to Remember

The unary operators `*` and `&` bind more tightly than arithmetic operators, so the assignment

`y = *ip + 1`

takes whatever `ip` points at, adds 1, and assigns the result to `y`, while

`*ip += 1`

increments what `ip` points to, as do

`++*ip`

and

`(*ip)++`

The parentheses are necessary in this last example; without them, the expression would increment `ip` instead of what it points to, **because unary operators like `*` and `++` associate right to left.**



#### `#include`

The "#include" directive brings in text from different files during compilation. #include is a very unintelligent and unstructured -- it just pastes in the text from the given file and continues compiling. The #include directive is used in the .h/.c file convention below which is used to satisfy the various constraints necessary to get prototypes correct.

```c
#include "foo.h"       // refers to a "user" foo.h file --
                       // in the originating directory for the compile
#include <foo.h>       // refers to a "system" foo.h file --
                       // in the compiler's directory somewhere
```

#### `#define`


```c
#include <stdio.h>
#define PI 3.1415
#define circleArea(r) (PI*r*r)

int main()
{
    int radius;
    float area;

    printf("Enter the radius: ");
    scanf("%d", &radius);
    area = circleArea(radius);
    printf("Area = %.2f", area);

    return 0;
}
```


#### `#include`

The "#include" directive brings in text from different files during compilation. #include is a very unintelligent and unstructured -- it just pastes in the text from the given file and continues compiling. The #include directive is used in the .h/.c file convention below which is used to satisfy the various constraints necessary to get prototypes correct.

```c
#include "foo.h"       // refers to a "user" foo.h file --
                       // in the originating directory for the compile
#include <foo.h>       // refers to a "system" foo.h file --
                       // in the compiler's directory somewhere
```

#### `#define`


```c
#include <stdio.h>
#define PI 3.1415
#define circleArea(r) (PI*r*r)

int main()
{
    int radius;
    float area;

    printf("Enter the radius: ");
    scanf("%d", &radius);
    area = circleArea(radius);
    printf("Area = %.2f", area);

    return 0;
}
```

***
#### FILE I/0

Declare a pointer of type file -      

    FILE *fptr;

Connecting to file – 

    ptr = fopen("fileopen","mode")

For Example: - 

```
fopen("E:\\cprogram\\newprogram.txt","w");
fopen("E:\\cprogram\\oldprogram.bin","rb");
```

Closing a file – 

    fclose(fptr); //fptr is the file pointer associated with file to be closed.


***
File Mode |	Meaning of Mode	| During Inexistence of file
----------|-----------------|----------------------------
R	      |Open for reading.|	If the file does not exist, fopen() returns NULL.
Rb	      |Open for reading in binary mode.|	If the file does not exist, fopen() returns NULL.
W	      |Open for writing.|If the file exists, its contents are overwritten. If the file does not exist, it will be created.
Wb	|Open for writing in binary mode.	|If the file exists, its contents are overwritten. If the file does not exist, it will be created.
A	|Open for append. i.e, Data is added to end of file.	|If the file does not exists, it will be created.
Ab	|Open for append in binary mode. i.e, Data is added to end of file.|	If the file does not exists, it will be created.
r+	|Open for both reading and writing.	|If the file does not exist, fopen() returns NULL.
rb+	|Open for both reading and writing in binary mode.	|If the file does not exist, fopen() returns NULL.
w+	|Open for both reading and writing.	|If the file exists, its contents are overwritten. If the file does not exist, it will be created.
wb+	|Open for both reading and writing in binary mode.	|If the file exists, its contents are overwritten. If the file does not exist, it will be created.
a+	|Open for both reading and appending.	|If the file does not exists, it will be created.
ab+	|Open for both reading and appending in binary mode.	|If the file does not exists, it will be created.




***
#### Example – writing to and reading from text file using `fprintf` and `fscanf`

```c
#include <stdio.h>
int main()
{
   int num;
   FILE *fptr;
   fptr = fopen("C:\\c\\program.txt","w");

   if(fptr == NULL)
   {
      printf("Error!");   
      exit(1);             
   }

   printf("Enter num: ");
   scanf("%d",&num);
   fprintf(fptr,"%d",num);
   
   fscanf(fptr,"%d", &num);
   printf("Value of n=%d", num);
   fclose(fptr);

   return 0;
}
```

#### Reading and writing to a binary file (using `fread()` and `fwrite()`)

Use – 

    fwrite(address_data,size_data,numbers_data,pointer_to_file);
    fread(address_data,size_data,numbers_data,pointer_to_file);

```c
#include <stdio.h>

struct threeNum
{
   int n1, n2, n3;
};

int main()
{
   int n;
   struct threeNum num;
   FILE *fptr;

   if ((fptr = fopen("C:\\C\\program.bin","wb")) == NULL){
       printf("Error! opening file");

       // Program exits if the file pointer returns NULL.
       exit(1);
   }

   for(n = 1; n < 5; ++n)
   {
      num.n1 = n;
      num.n2 = 5*n;
      num.n3 = 5*n + 1;
      fwrite(&num, sizeof(struct threeNum), 1, fptr); 
   }
   fclose(fptr); 
  
   return 0;
}
```

##### Program for reading from binary file – 

```c
#include <stdio.h>

struct threeNum
{
   int n1, n2, n3;
};

int main()
{
   int n;
   struct threeNum num;
   FILE *fptr;

   if ((fptr = fopen("C:\\C\\program.bin","rb")) == NULL){
       printf("Error! opening file");

       // Program exits if the file pointer returns NULL.
       exit(1);
   }

   for(n = 1; n < 5; ++n)
   {
      fread(&num, sizeof(struct threeNum), 1, fptr); 
      printf("n1: %d\tn2: %d\tn3: %d\t", num.n1, num.n2, num.n3);
   }
   fclose(fptr); 
  
   return 0;
}
```

***
##### Getting data using `fseek()` –

    fseek(FILE * stream, long int offset, int whence)

**Note -**  didn’t understand the output of following program.

```c
#include <stdio.h>

struct threeNum
{
   int n1, n2, n3;
};

int main()
{
   int n;
   struct threeNum num;
   FILE *fptr;

   if ((fptr = fopen("C:\\c\\program.bin","rb")) == NULL){
       printf("Error! opening file");
       // Program exits if the file pointer returns NULL.
       exit(1);
   }
   
   // Moves the cursor to the end of the file
   fseek(fptr, sizeof(struct threeNum), SEEK_END);

   for(n = 1; n < 5; ++n)
   {
      fread(&num, sizeof(struct threeNum), 1, fptr); 
      printf("n1: %d\tn2: %d\tn3: %d\t", num.n1, num.n2, num.n3);
   }
   fclose(fptr); 
  
   return 0;

}
```

### `system` function

In [14]:
#include <stdio.h>
int main()
{
    system("calc"); //windows specific`
    system("date"); /*displays result only when CTRL+C pressed*/
}

/var/folders/d4/z0x5s3410lj8px16wr2m5y200000gp/T/tmpmo_jgag5.c:4:5: warning: implicit declaration of function 'system' is invalid in C99 [-Wimplicit-function-declaration]
    system("calc");
    ^
1 warning generated.
sh: calc: command not found


Tue Apr 28 20:13:39 IST 2020


***

#### C program to display its own source code using `__FILE__`

```c
#include <stdio.h>
int main() {
    FILE *fp;
    char c;
    fp = fopen(__FILE__,"r");
    do {
         c = getc(fp);
         putchar(c);
    }
    while(c != EOF);
    fclose(fp);
    return 0;
}
```

In [22]:
#include <stdio.h>
int main()
{
   printf("Current time: %s\n",__TIME__);   //calculate the current time
   printf("Current date: %s\n",__DATE__);
   printf("Current file: %s\n",__FILE__);
   //printf("Current time: %s\n",__LINE__);   not working

}

/var/folders/d4/z0x5s3410lj8px16wr2m5y200000gp/T/tmp9bckg4u7.c:7:32: warning: format specifies type 'char *' but the argument has type 'int' [-Wformat]
   printf("Current time: %s\n",__LINE__);   
                         ~~    ^~~~~~~~
                         %d
<scratch space>:489:1: note: expanded from here
7
^
1 warning generated.


Current time: 20:18:39
Current date: Apr 28 2020
Current file: /var/folders/d4/z0x5s3410lj8px16wr2m5y200000gp/T/tmp9bckg4u7.c


[C kernel] Executable exited with code -11